In [ ]:
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load your dataset (e.g., from a folder structure)
def load_images_and_labels(image_dir, img_size, categories):
    images, labels = [], []
    for category in categories:
        path = os.path.join(image_dir, category)
        class_num = categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
                img_array = cv2.resize(img_array, (img_size, img_size))
                images.append(img_array)
                labels.append(class_num)
            except Exception as e:
                pass
    return np.array(images), np.array(labels)

# Define the categories based on HAM10000 dataset (7 disease categories)
categories = ['nv', 'mel', 'bkl', 'bcc', 'akiec', 'vasc', 'df']

# Load images and labels
IMG_SIZE = 128
image_dir = 'path_to_your_HAM10000_data'
X, y = load_images_and_labels(image_dir, IMG_SIZE, categories)

# Normalize and preprocess the data
X = X / 255.0
y = to_categorical(y, num_classes=len(categories))

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D(2, 2),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    model.compile(optimizer='adam', 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy', 
                           tf.keras.metrics.AUC(name='auc'), 
                           tf.keras.metrics.Precision(name='precision'), 
                           tf.keras.metrics.Recall(name='recall')])
    return model

# Build the model
model = build_model((IMG_SIZE, IMG_SIZE, 3), len(categories))
model.summary()


In [ ]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

# Fit the model
history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    epochs=30, 
                    validation_data=(X_test, y_test))


In [ ]:
from sklearn.metrics import classification_report, f1_score, average_precision_score

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Classification report
report = classification_report(y_true, y_pred_classes, target_names=categories)
print(report)

# Compute F1 Score
f1 = f1_score(y_true, y_pred_classes, average='weighted')
print(f"F1 Score: {f1}")

# Compute mean Average Precision (mAP)
map_score = average_precision_score(y_test, y_pred, average='weighted')
print(f"Mean Average Precision (mAP): {map_score}")


In [ ]:
import matplotlib.pyplot as plt

# Plot accuracy and loss
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()
